# Hypercube

### Table of Content
1. Overview
2. Definitions and Basics
3. Rolling
4. Memory-Mapped Hypercube
5. Cube Math and Vectorized Cube Math
6. Python Integration and Interoperability with `numpy`
7. Translating `numpy` code to Java
8. Benchmarks and Performance

## Overview
`hypercube` is an `ndarray`-like package for Java, designed to offer a representation for `numpy` `ndarray`s that is feature-rich in numerical computations. Furthermore, hypercubes are designed to be interoperable with `numpy`, as we'll show in the examples. Additionally, this package can be used to directly translate `numpy` code into Java. E.g., you can prototype in Python but, if you have a codebase which is using Java for the heavy computations, then Hypercube can aid the development process.

As part of the `hypercube` package, the `CubeMath` library is designed to offer many of `numpy`'s math operations for Hypercubes. As part of this effort, we have also developed `VectorizedCubeMath`, an experimental library, that offers SIMD-accelerated operations through [the Vector API](https://openjdk.org/jeps/426). It's worth making note here that [the Vector API](https://openjdk.org/jeps/426), as of 08/18/2023, is an incubator module in Java, and so it should be regarded as highly experimental.

In this notebook, we'll be using hypercubes in Python (through PJRmi) for ease of demonstration. We'll be covering:
- Hypercubes and `ndarray` operations on them
- `CubeMath` and math operations on hypercubes
- `VectorizedCubeMath` (experimental library)
- Performance and benchmarks

### Let's dive in!

Note that we're passing the `includeVectorized` parameter to include `VectorizedCubeMath` in our build. By default, all Vectorized code is hidden from the build, since it requires Java 17 and we don't want to force users there unless we have to.

Assuming you are in the PJRmi directory and only have one wheel file, you can run something like:

```
./gradlew wheel -PincludeVectorized
unzip -f python/pjrmi-*.whl
```
Note: If some of the cells in the below do not work, you may need to add the directory containing the wheel (e.g. `pjrmi/python`) to the `PYTHONPATH`. This can occur when the global PJRmi version differs from the one we just built.

Now we'll setup a PJRmi connection. Note that we're:
- Using shared memory passing (for our benchmarks)
- Enabling the incubated Vector API module in Java to be able to use `VectorizedCubeMath`

##### Note on Fine-Tuning Performance
It's worth noting that both of the cube math libraries support multithreaded processing, as well as efficient loop unrolling. By default, both libraries use a staging size of 128 for better loop unrolling and 4 threads for multithreaded processing, which are only enabled after a rough threshold to avoid the overhead of multithreading. More formally:
- Staging size: The number of elements to process "in bulk" (by default 128).
- Number of threads: The number of threads to use when multithreading (by default 4).
- Multithreading threshold: The cube size at which Cube Math will enable multithreaded processing (by default 131,072).

These parameters can all be arbitrarily set to fine-tune the performance of the cube math libraries to specific environments or machines. To do so, we can pass the following corresponding Java properties when initializing our Java connection.
- `com.deshaw.hypercube.cubemath.stagingSize` -- The number of elements to stage when processing operations in bulk.
- `com.deshaw.hypercube.cubemath.threadingThreshold` -- The cube size threshold at which VectorizedCubeMath will use multithreading.
- `com.deshaw.hypercube.cubemath.numThreads` -- The number of threads used for multithreaded processing.

In [ ]:
# What we'll need for this
import numpy as np, numpy
import pjrmi
import time
import timeit

# Get a new instance.
#
# Here the application_args are just to allow the server to have
# multiple threads. This is only really needed if you intend to
# use callbacks, which we will in our examples later below.
#
# We also pipe the stdout and stderr of the Java process to
# /dev/null by means of setting the filehandles to None.
cx = pjrmi.connect_to_child_jvm(stdout=None,
                                stderr=None,
                                application_args=('num_workers=4',),
                                use_shm_arg_passing=True,
                                java_args=['--add-modules', 'jdk.incubator.vector'])

### Hypercube Class Definitions

Here we'll define the relevant Hypercube classes in Python, to be able to play around with them. We'll also define our two cube math libraries.

In [ ]:
# Defining all the relevant Hypercube classes
Dimension              = cx.class_for_name('com.deshaw.hypercube.Dimension')
Hypercube              = cx.class_for_name('com.deshaw.hypercube.Hypercube')
BooleanBitSetHypercube = cx.class_for_name('com.deshaw.hypercube.BooleanBitSetHypercube')
DoubleArrayHypercube   = cx.class_for_name('com.deshaw.hypercube.DoubleArrayHypercube')
FloatArrayHypercube    = cx.class_for_name('com.deshaw.hypercube.FloatArrayHypercube')
IntegerArrayHypercube  = cx.class_for_name('com.deshaw.hypercube.IntegerArrayHypercube')
LongArrayHypercube     = cx.class_for_name('com.deshaw.hypercube.LongArrayHypercube')
DoubleMappedHypercube  = cx.class_for_name('com.deshaw.hypercube.DoubleMappedHypercube')
DoubleSparseHypercube  = cx.class_for_name('com.deshaw.hypercube.DoubleSparseHypercube')

# Defining our two Cube Math implementations and aliases for them
CubeMath           = cm  = cx.class_for_name('com.deshaw.hypercube.CubeMath')
VectorizedCubeMath = vcm = cx.class_for_name('com.deshaw.hypercube.VectorizedCubeMath')

Let's create an integer hypercube to work with.

In [ ]:
cube = IntegerArrayHypercube(Dimension.of(10))
cube

Now let's populate our cube with some values. We can use the `fromFlattened()` method for this. As the name suggests, using this method we can _unflatten_ an array of data into our cube. We can use `numpy` arrays, lists or any other iterable for this.

In [ ]:
cube.fromFlattened(np.arange(10, dtype = np.int32))
print(cube)
cube.fromFlattened(range(10, 0, -1))
print(cube)

We can also use most Python syntax on hypercubes. Here's an example of a few things we can do:

In [ ]:
# We can call magic functions on cubes
print(f"Length of the cube is {len(cube)}.")
for i,e in enumerate(cube):
    print(f"The {i}-th element of the cube is {e}.")

# We can slice cubes
subcube = cube[3:7]
print("Subcube:")
print(subcube)

# We can use Python's get/set syntax
subcube[0] = 10print(cube[3]) # Unsurprisingly, the change is reflected in the original cube.

### Rolling

Much like `ndarray`s, hypercubes support both flat rolling, and rolling across certain axes. This allows us to use cubes in a streaming data idiom.

For example, if the primary axis is time, then we could roll the cube along that axis by one, pushing data back such that the oldest data wraps to the head, and then overwrite that data with the new data:

    data                        [5, 4, 3, 2, 1]
    data.roll(1, axis = 0)  =>  [1, 5, 4, 3, 2]
    data[0] = 6             =>  [6, 5, 4, 3, 2] 

By doing so, we can use the same code in a static (non-realtime) context, as well as a realtime one.

In [ ]:
# First let's reshape the cube, so we have multiple dimensions to work with.
reshaped = cube.reshape((2, 5))
print("Reshaped:")
print(reshaped)

# Now shift (i.e., "flat roll") the cube.
shifted = reshaped.roll(2)
print("Shifted:")
print(shifted)

# Now let's try rolling the cube across both axes.
rolled = reshaped.roll((1, 2))
print("Rolled:")
print(rolled)

# We can also roll a cube by specifying a specific dimension
print(reshaped.roll(2, axis=1))

### Memory-Mapped Hypercube

Hypercube also offers a class of memory-mapped cubes that store their elements in a memory-mapped file. This memory-mapped file is readable by `numpy`'s memory-mapped arrays, which allows for seamless data transfer between Java and Python. For example, you can simultaneously work on the same `ndarray` both from a Java and a Python client, without even using PJRmi.

We can see an example of this seamless transfer between these cubes and arrays below:

In [ ]:
marray = numpy.memmap('/dev/shm/example.dat', dtype = numpy.float64, mode = 'w+', shape = (3,3), order = 'C')
mcube  = DoubleMappedHypercube('/dev/shm/example.dat', Dimension.of((3, 3)))

print("Memory-mapped array:")
print(marray)
print("Memory-mapped cube:")
print(mcube)

# Let's make a change in our array
marray += 1

# Let's see the change reflected in our cube
print("Updated memory-mapped cube:")
print(mcube)

# Now let's make a change to our cube
CubeMath.negative(mcube, mcube)

# Let's see the change reflected in our array
print("Updated memory-mapped array:")
print(marray)

### Sparse Hypercube

For high-dimensional data which isn't dense it may be desirable to have a more memory efficient backing store. The sparse cubes provide this support.

In [ ]:
sparse = DoubleSparseHypercube(Dimension.of((4, 4)))
k = 0
for i in range(0, 4):
    for j in range(0, 4):
        if ((i + j) % 2) == 0:
            sparse[i,j] = k
            k += 1
print("Sparse:")
print(sparse)
print("Sum: ", CubeMath.nansum(sparse))

Now that we've established the basics, let's get into the fun bits!

### Cube Math

Hypercube offers many of `numpy`'s `ndarray` operations for cubes, through the `CubeMath` and `VectorizedCubeMath` libraries. We have added support for some of the most commonly used `numpy` `ndarray` operations as a start, with more operations to be supported soon. Now, while these two libraries use different internal implementations for their operations, they both provide the exact same API to the users, and can be used interchangeably. Here we'll only show the examples on CubeMath for the sake of brevity.

As of 08/23/2023, the full list of supported operations and data types in CubeMath can be found in the tables below:

| Arithmetic | Arithmetic | Bitwise | Comparative | Comparative | Math  | Math  | Math  | Reductive | Reductive | Misc.     |
|:----------:|:----------:|:-------:|:-----------:|:-----------:|:-----:|:-----:|:-----:|:---------:|:---------:|:---------:|
| add        | modulo     | and     | =           | !=          | sin   | cos   | tan   | sum       | nansum    | extract   |
| subtract   | power      | or      | <           | >           | sinh  | cosh  | tanh  | min       | max       | type cast |
| multiply   | negative   | xor     | <=          | >=          | exp   | log   | log10 | any       | all       | copy      |
| divide     | abs        | not     |             |             | floor | round | ceil  |           |           | broadcast |

VectorizedCubeMath supports the same operations and data types as CubeMath. However, vectorized implementation is not supported for certain operations (marked in red in the table below). VectorizedCubeMath uses the standard implementation for these operations.

| Arithmetic | Arithmetic | Bitwise | Comparative | Comparative | Math  | Math  | Math  | Reductive | Reductive |
|:----------:|:----------:|:-------:|:-----------:|:-----------:|:-----:|:-----:|:-----:|:---------:|:---------:|
| add        |            | and     | =           | !=          | sin   | cos   | tan   | sum       | nansum    |
| subtract   | power      | or      | <           | >           | sinh  | cosh  | tanh  | min       | max       |
| multiply   | negative   | xor     | <=          | >=          | exp   | log   | log10 | any       | all       |
| divide     | abs        | not     |

Now, let's see some of these operations in action:

#### Basic math operations

In [ ]:
# Again, don't forget to properly cast your numbers in Python
print("Add with a value:\n",      CubeMath.add  (cube, 10))
print("Add with a cube:\n",       CubeMath.add  (cube, cube))
print("Xor with a cube:\n",       CubeMath.xor  (cube, cube))
print("Equality with a value:\n", CubeMath.equal(cube, 10))
print("Equality with a cube:\n",  CubeMath.equal(cube, cube))

#### Reductive operations

In [ ]:
print("Min value:\n", CubeMath.min0d(cube))
print("Max value:\n", CubeMath.max0d(cube))
print("Sum:\n", CubeMath.sum0d(cube))

# This one doesn't really make sense for integers, but you get the idea.
print("NanSum:\n", CubeMath.nansum0d(cube))

#### Boolean selection and population count

You can mix and match to get more complicated expressions too.

In [ ]:
print("How many values in the cube are greater than 6?\n", CubeMath.popcount(CubeMath.greater(cube, np.int32(6))))

# Boolean extraction (selection)
print("The elements that are smaller than 5:\n", CubeMath.extract(CubeMath.less(cube, np.int32(5)), cube))

##### Copying and casting
CubeMath also allows you to get casted copied of cubes in different data types. We can also just copy a given cube without casting it to other data types.

In [ ]:
double_cube = CubeMath.toDoubleHypercube(cube)
print("Double Hypercube:\n", double_cube)

copy_cube = CubeMath.copy(cube)
print("Copy:\n", copy_cube)

#### Math functions

CubeMath also supports your favorite math functions, from trigonometric functions, to basic utility functions like `floor()`.

Note that certain math functions (like trigonometric ones) are only supported for floating-point data types. You can, however, cast your integer cubes (just we did like above) before applying your favorite math functions.

In [ ]:
print("Sin:\n",   CubeMath.sin(double_cube))
print("Cos:\n",   CubeMath.cos(double_cube))
print("Tanh:\n",  CubeMath.tanh(double_cube))
print("Exp:\n",   CubeMath.exp(double_cube))
print("Log:\n",   CubeMath.log(double_cube))
print("Log10:\n", CubeMath.log10(double_cube))

### Python Integration and interoperability with `numpy`

Hypercubes can almost always be treated `ndarray`s in Python and so `ndarray` syntax and functions apply to them.

In [ ]:
print(cube)
print(cube - cube)
print(cube ** 2)
print(cube / 2)
print(cube // 2)
print(cube == 10)
print(cube[cube > 6])

In [ ]:
# We can make numpy arrays from cubes.
array = np.array(cube)
print("Array:\n", type(array), array)

# We can use numpy functions on cubes.
print("`numpy` sum of cube:\n", np.sum(cube))

# We can use CubeMath functions on numpy arrays.
print("CubeMath sum of `numpy` array:\n", CubeMath.sum0d(array))

# We can call operations on numpy arrays and cubes together, both from numpy and CubeMath.
print("`numpy` equal:\n", array == cube)
print("CubeMath equal:\n", CubeMath.equal(array, cube))

Note that hypercube instances do not support all `numpy` array methods as attributes. You can, however, call the static `numpy` version of these functions with your cube as an argument. Let's take a look at the following example to see what you can and can not expect from hypercube:

In [ ]:
reshaped_array = np.array(reshaped)

print("Our previously reshaped cube as an array:")
print(reshaped_array)
print("Now with the axes swapped:")
print(reshaped_array.swapaxes(0, 1))

# Now try swapping the cube's axes through the static np.swapaxes method
print("Swapping the cube's axes:")
print(np.swapaxes(reshaped, 0, 1))

What happened?

We see that `reshaped.swapaxes(0, 1)` throws an error since our cube object has no attribute called `swapaxes` (which is as expected). However, this doesn't stop us from calling the static `np.swapaxes` method on our cube.

### Translating `numpy` to Java

At times, we might find the need or the desire to translate code written in `numpy` to Java. Here, we are going to show you the very broad formula for doing this translation. Depending on your specific needs, you may need to use Hypercube or CubeMath differently, but these general instructions will hopefully provide useful insights for the users:

1. Magic methods: While Python supports magic dunder functions like `len()`, Java lacks such a feature and so hypercubes cannot use the same syntax in Java (this is also the case for operator overloading, e.g. doing `array[indices]`). However, Hypercube implements similar methods that can be used to achieve the same functionality (`cube.getObj(indices)` and `cube.get(indices)` as opposed to `array[indices]`). Alternatively, you can explicitly call the corresponding dunder methods on hypercubes (e.g., `cube.__getitem__(indices)`), although we do not recommend this behavior.
2. It is worth noting here that due to how Java's type system works, primitive hypercubes (e.g., `IntegerHypercube` etc.) offer both object methods, and primitive ones for many of their operations (e.g., `cube.getObj(indices)` and `cube.get(indices)` as mentioned above). While all these method work as expected, we encourage using the latter for primitive hypercubes, since methods that deal with objects tend to make for a lousy performance, significantly slower than their primitive counterparts.
3. Static methods instead of member methods: While `numpy` provides many of its operations as instance methods, hypercubes (as of 08/24/2023) do not do the same. For example, while you are able to write `array.sum()`, using `cube.sum()` will not work. However, Hypercube offers `ndarray` math operations as part of the CubeMath library that can be used as static method. For example, you would do the same as above by calling `CubeMath.sum0d(cube)` (note that `numpy` supports a similar static notation, i.e. `numpy.sum(array)`).
4. Type casting: CubeMath and `numpy` differ in the way they handle type-casting for certain operations. For example, `numpy` always converts integer arrays to real arrays before performing certain math operations like sin and cos. CubeMath, on the other hand, was designed to loosely follow Java's typing system, meaning that floating-point operations (like sin, cos) are not supported on integer cubes. When translating `numpy` code to Java, think about how you want to cubes to be casted, before certain operations. Conveniently, CubeMath supports casting methods that can be used to do this.

### Benchmarks and Performance

Now finally, let's dive into the exciting realm of performance and see how CubeMath and VectorizedCubeMath compare to each other, as well as to `numpy`.

It's worth noting here that we're not trying to outperform or replace `numpy`, but rather try to mainly show _comparable_ performance.

#### Technical note on vectorization:
All supported operations in VectorizedCubeMath use the Vector API internally, while none in CubeMath do. However, certain operations in CubeMath can be auto-vectorized by compilers and thus be translated into SIMD instructions. This is one of the reasons that for certain operations, CubeMath performs similarly or even slightly faster than VectorizedCubeMath.

#### Technical note on multithreaded processing:
By default CubeMath and VectorizedCubeMath use 4 threads when using multithreaded processing (which is the case here).

However, It's more difficult to reason about `numpy`'s multithreading and the exact number of threads it uses. Internally, `numpy` uses multiple multithreaded math libraries (e.g., Intel MKL and LAPACK) that work independently and use a different number of threads. This is different from the PJRmi threading model. While it is possible to individually limit the number of used threads on the internal libraries, configuring the number of threads is idiosyncratic to the various underlying backend libraries.

Take note that the performance of these libraries are highly dependant on their configurations. For more accurate comparisons, it is very important to make sure you configure CubeMath and `numpy`'s multithreaded processing to use similar resources. For more information on the internal backend libraries that your `numpy` is using, you can run the `numpy.show_config()` command that provides all the relevant backend details, as well as supported SIMD instructions. For more information on the multithreading schemes of the individual backend libraries, consult the libraries documentations.

#### Methodology

For our benchmarks, we'll be using IPython's own magic `%timeit` function on randomly generated 64-bit floating-point arrays of 100 million elements. This large array size is deliberately chosen to reduce the relative noise of the benchmark instruments, as well as PJRmi communication between Python and Java.

You should test the performance of these libraries with your desired functions, data types and array sizes.

#### Before we begin...
Here we define some utility methods for creating `numpy` arrays and hypercubes, plus some _type abstractions_ for the sake of easier testing in Python. Really, what we're doing here is abstracting the internal hypercube implementations, for easier cube initialization in Python.

In [ ]:
# Utility method to make a `numpy` array of a given size and data type
def make_array(size = 1e6, dtype = np.float64):
    if dtype in [np.float32, np.float64]:
        return np.array(np.random.rand(int(size)), dtype=dtype)
    elif dtype == np.int32:
        return np.array(np.random.rand(int(size)) * 1e9, dtype=dtype)
    elif dtype == np.int64:
        return np.array(np.random.rand(int(size)) * 1e18, dtype=dtype)
    else:
        return np.array(np.random.rand(int(size)), dtype=np.float32) > 0.5

# Method to make a Hypercube out of a given `numpy` array, or a given size and data type
def make_cube(size = 1e6, dtype = np.float64, arr = None):
    if not isinstance(arr, numpy.ndarray):
        arr = make_array(size, dtype)
    return CubeMath.copy(arr)

#### Benchmarking

Now let's dive into the exciting part!

In [ ]:
size = 1e8
dtype = np.float64

# We use different cubes for CubeMath and VectorizedCubeMath to avoid caching benefits for one against the other.
a = make_array(size, dtype)
b = make_cube(size, dtype)
c = make_cube(size, dtype)

First let's test some basic math operations.

Note that we'll be using in-place computations throughout to avoid memory allocation noise in our timings.

In [ ]:
%timeit np.add(a, a, a)
%timeit CubeMath.add(b, b, b)
%timeit VectorizedCubeMath.add(c, c, c)

In [ ]:
%timeit np.exp(a, a)
%timeit CubeMath.exp(b, b)
%timeit VectorizedCubeMath.exp(c, c)

Let's repopulate our array and cubes. Since we are using in-place operations, results can quickly propagate into very large/small numbers for certain operations (e.g., `exp`). We _reset_ our array and cubes to make sure we're passing a clean copy to the next operations.

In [ ]:
a = make_array(size, dtype)
b = make_cube(size, dtype)
c = make_cube(size, dtype)

In [ ]:
%timeit np.sin(a, a)
%timeit CubeMath.sin(b, b)
%timeit VectorizedCubeMath.sin(c, c)

In [ ]:
%timeit np.tanh(a, a)
%timeit CubeMath.tanh(b, b)
%timeit VectorizedCubeMath.tanh(c, c)

In [ ]:
%timeit np.sum(a)
%timeit CubeMath.sum0d(b)
%timeit VectorizedCubeMath.sum0d(c)

In [ ]:
%timeit np.nansum(a)
%timeit CubeMath.nansum0d(b)
%timeit VectorizedCubeMath.nansum0d(c)

In [ ]:
%timeit np.min(a)
%timeit CubeMath.min0d(b)
%timeit VectorizedCubeMath.min0d(c)

Again, let's repopulate our array and cubes here to make sure we're using numbers between [0, 1] so that the result of `power` always stays in the same range.

In [ ]:
a = make_array(size, dtype)
b = make_cube(size, dtype)
c = make_cube(size, dtype)

In [ ]:
%timeit np.power(a, a, a)
%timeit CubeMath.power(b, b, b)
%timeit VectorizedCubeMath.power(c, c, c)

### Discussion on the Benchmark Results

While it is unwise to generalize the performance of these libraries within certain environments to the others, it is useful to hypothesize why the libraries behave in a certain way for certain operations. That said, we will not be going into too much detail here, as our main goal is to show CubeMath and VectorizedCubeMath's competitive performance (and that they are not significantly slow compared to `numpy`).

1. Basic Arithmetic Operations (e.g., +, -, =, etc.):
The performance of CubeMath is expected to be similar to VectorizedCubeMath for these operations. This is because compilers are able to auto-vectorize these operations in CubeMath, resulting in SIMD performance.
2. Complex Math Operations (e.g., sin, cos, exp, power, etc.):
VectorizedCubeMath outperforms CubeMath (and at times `numpy`) in these operations because it uses specialized SIMD-instructions, as well as a highly optimized algorithm, written in native assembly and C. The exact performance of `numpy` will depend on the underlying backend libraries, and its configuration.
3. Reductive operations (e.g., sum, nansum, etc.):
VectorizedCubeMath and CubeMath perform similarly for reductive operations, with CubeMath being faster for some (e.g., min) and VectorizedCubeMath being faster in others (e.g., sum). Notably, CubeMath and VectorizedCubeMath tend to outperform `numpy` in nansum, since the latter uses `sum()` with a masked ndarray.

It's worth noting once again that depending on the available hardware and the specific configurations, these libraries may perform differently.


### Wrapping up...

This was just a very quick introduction to some of the things we can do with Hypercube. The general rule of thumb is that: if you expect it to work, then it probably should.
